# Load all the files

In [1]:
import numpy as np
import pandas as pd

chald = pd.read_csv('challenge_data.csv')
dftrain = pd.read_csv('train.csv')
dftest = pd.read_csv('test.csv')
samp_sub = pd.read_csv('sample_submission_J0OjXLi_DDt3uQN.csv')


In [2]:
print chald.shape

# print np.isnan(chald.iloc[:,0].unique())

(5606, 9)


In [3]:
import matplotlib.pyplot as plt
ts = chald.challenge_ID.apply(lambda y: int(y[2:]))
print chald.head(5)
lst=[]
for i in range(8):
    print chald.columns[i+1]
    print chald.iloc[:,i+1].unique().shape
    lst.append(chald.iloc[:,i+1].unique().shape[0])
    print chald.iloc[:,i+1].unique()

  challenge_ID  programming_language challenge_series_ID  total_submissions  \
0      CI23478                     2              SI2445               37.0   
1      CI23479                     2              SI2435               48.0   
2      CI23480                     1              SI2435               15.0   
3      CI23481                     1              SI2710              236.0   
4      CI23482                     2              SI2440              137.0   

  publish_date author_ID author_gender author_org_ID  category_id  
0   06-05-2006  AI563576             M     AOI100001          NaN  
1   17-10-2002  AI563577             M     AOI100002         32.0  
2   16-10-2002  AI563578             M     AOI100003          NaN  
3   19-09-2003  AI563579             M     AOI100004         70.0  
4   21-03-2002  AI563580             M     AOI100005          NaN  
programming_language
(3,)
[2 1 3]
challenge_series_ID
(436,)
['SI2445' 'SI2435' 'SI2710' 'SI2440' 'SI2439' 'SI2438' '

### Separate the INPUT challange_ID s from the OUTPUT challange_ID s

##### 1, For the TRAIN SET

In [4]:
df = pd.DataFrame(dftrain.groupby(['user_id'])['challenge'].apply(list))
%time Chin = pd.DataFrame(df.apply(lambda y: y.iloc[0][0:10],axis=1))
%time Chout = pd.DataFrame(df.apply(lambda y: y.iloc[0][10:],axis=1))
print df.iloc[0].iloc[0]
#df = pd.DataFrame(dftrain.groupby(['user_id'])['challenge'])
Chin.columns = ['chall_ids']
Chout.columns = ['chall_ids']


CPU times: user 1.76 s, sys: 24.4 ms, total: 1.79 s
Wall time: 1.78 s
CPU times: user 1.87 s, sys: 196 ms, total: 2.07 s
Wall time: 1.92 s
['CI23714', 'CI23855', 'CI24917', 'CI23663', 'CI23933', 'CI25135', 'CI23975', 'CI25126', 'CI24915', 'CI24957', 'CI24958', 'CI23667', 'CI23691']


##### 1, For the TEST SET

In [6]:
dftest.head()
Chinvalid = pd.DataFrame(dftest.groupby(['user_id'])['challenge'].apply(list))
Chinvalid.head()
Chinvalid.columns = ['chall_ids']
Chinvalid.head()


,chall_ids
user_id,
4577,"[CI23855, CI23933, CI24917, CI24915, CI23714, ..."
4578,"[CI23663, CI23855, CI24917, CI23933, CI23975, ..."
4579,"[CI26939, CI26940, CI26941, CI26942, CI26943, ..."
4583,"[CI23663, CI23855, CI23975, CI23714, CI23848, ..."
4584,"[CI23855, CI23975, CI25135, CI23848, CI23714, ..."


#### Clean the challange set (with input features) from NaNs and convert values into integers

In [7]:
def clean_dataset(chald):
    chald2 = chald.copy()
    chald2 = chald2.set_index('challenge_ID')
    print chald2.head()

    # here I clean the NaN s from the challenge_series_ID columns and then convert them into INTEGERS
    chald2.challenge_series_ID[chald2.challenge_series_ID.isna()] = ["SI0"]
    chald2.challenge_series_ID = chald2.challenge_series_ID.apply(lambda y: int(y[2:]))
    # sum(chald2.challenge_series_ID[chald2.challenge_series_ID.isna()])

    # here I clean the NaN s from the author_ID columns and then convert them into INTEGERS
    chald2.author_ID[chald2.author_ID.isna()] = ['AI0']
    chald2.author_ID = chald2.author_ID.apply(lambda y: int(y[2:]))
    chald2.head(10)

    chald2.author_org_ID[chald2.author_org_ID.isna()] = ['AOI0']
    chald2.author_org_ID = chald2.author_org_ID.apply(lambda y: int(y[3:]))

    chald2.category_id[chald2.category_id.isna()] = [0]
    chald2.category_id = chald2.category_id.astype(int)

    chald2.total_submissions[chald2.total_submissions.isna()] = [0]
    chald2.total_submissions = chald2.total_submissions.astype(int)

    nan_for_gender = chald2.author_gender.isna()
    chald2.author_gender = chald2.author_gender.apply(lambda y: int(y=='M'))
    chald2.author_gender[nan_for_gender] = -1
    chald2.author_gender = chald2.author_gender+1
    
    return chald2
chald2 = clean_dataset(chald)
chald2.head(10)

              programming_language challenge_series_ID  total_submissions  \
challenge_ID                                                                
CI23478                          2              SI2445               37.0   
CI23479                          2              SI2435               48.0   
CI23480                          1              SI2435               15.0   
CI23481                          1              SI2710              236.0   
CI23482                          2              SI2440              137.0   

             publish_date author_ID author_gender author_org_ID  category_id  
challenge_ID                                                                  
CI23478        06-05-2006  AI563576             M     AOI100001          NaN  
CI23479        17-10-2002  AI563577             M     AOI100002         32.0  
CI23480        16-10-2002  AI563578             M     AOI100003          NaN  
CI23481        19-09-2003  AI563579             M     AOI100004  

/Users/zsombor/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zsombor/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zsombor/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/zsombor/anaconda/envs/py27/lib/python2.7/site-packages/ipykernel/__main__.py:19: Setting

,programming_language,challenge_series_ID,total_submissions,publish_date,author_ID,author_gender,author_org_ID,category_id
challenge_ID,,,,,,,,
CI23478,2,2445,37,06-05-2006,563576,2,100001,0
CI23479,2,2435,48,17-10-2002,563577,2,100002,32
CI23480,1,2435,15,16-10-2002,563578,2,100003,0
CI23481,1,2710,236,19-09-2003,563579,2,100004,70
CI23482,2,2440,137,21-03-2002,563580,2,100005,0
CI23483,2,2445,1434,06-05-2006,0,0,0,70
CI23484,1,2440,509,22-03-2006,563582,1,100007,32
CI23485,1,2435,287,17-10-2002,563583,1,100008,23
CI23486,2,2440,19,20-03-2002,563584,2,100009,0


#### Obtain one of the features that I want to use (mean total submission)

In [ ]:
print Chin.size
## creat the input array with the first feature, mean total submission for each user
%time inputs = Chin.apply([lambda y: chald2.total_submissions.loc[y[0:]].mean()])
inputs.columns = inputs.columns.droplevel()
inputs.columns = ['mean_tot_submission']
inputs.head()


## creat the output array with the first feature, mean total submission for each user
%time outputs = Chout.apply([lambda y: chald2.total_submissions.loc[y[0:]].mean()])
outputs.columns = outputs.columns.droplevel()
outputs.columns = ['mean_tot_submission']
outputs.head()

In [477]:
## creat the input array with the first feature, mean total submission for each user
%time tmp = Chin.apply([lambda y: chald2.total_submissions.loc[y[0:]].max()])
tmp.columns = tmp.columns.droplevel()
tmp.columns = ['mean_tot_submission']
inputs['max_tot_submission'] = tmp['mean_tot_submission']
print inputs.head()

## creat the output array with the first feature, mean total submission for each user
%time tmp = Chout.apply([lambda y: chald2.total_submissions.loc[y[0:]].max()])
tmp.columns = tmp.columns.droplevel()
tmp.columns = ['mean_tot_submission']
outputs['max_tot_submission'] = tmp['mean_tot_submission']
outputs.head()

CPU times: user 39.4 s, sys: 286 ms, total: 39.7 s
Wall time: 39.6 s
         mean_tot_submission  max_tot_submission
user_id                                         
4576                 13165.7             43409.0
4580                  9019.7             20993.0
4581                  3547.0              6968.0
4582                 13315.8             43409.0
4585                 13538.4             43409.0
CPU times: user 40.2 s, sys: 244 ms, total: 40.4 s
Wall time: 40.3 s


<bound method DataFrame.head of          mean_tot_submission  max_tot_submission
user_id                                         
4576             5215.000000              6842.0
4580             6129.000000              7389.0
4581              831.000000              1096.0
4582             4158.000000              5446.0
4585             4639.666667              6814.0
4587            11227.000000             20993.0
4590             4611.333333              7731.0
4591             9900.666667             20674.0
4592             4491.333333              6842.0
4593              874.000000              1096.0
4596             2666.666667              6012.0
4602             3342.000000              6012.0
4604            10820.000000             15086.0
4606            11113.000000             20993.0
4609             4633.666667              9204.0
4611             1474.666667              2413.0
4612            16994.333333             43409.0
4615              580.666667         

In [ ]:
# print outputs.mean_tot_submission
print inputs.shape
plt.plot(inputs.mean_tot_submission/10, outputs.mean_tot_submission/3,'.')
plt.show()


In [ ]:
plt.plot(inputs.max_tot_submission/10, outputs.mean_tot_submission/3,'.')
plt.show()

# Create the output labels from challenge_ID s (train set)

In [8]:
chald2['index'] = range(chald2.shape[0])
a = Chout
s = a.apply(lambda x: pd.Series(x[0]),axis=1).stack().reset_index(level=1, drop=True)
output_index = chald2['index'].loc[s]
output_index.head()

challenge_ID
CI24958    1480
CI23667     189
CI23691     213
CI24915    1437
CI25727    2249
Name: index, dtype: int64

In [9]:
# from sklearn import preprocessing
# lb = preprocessing.LabelBinarizer()
# lb.fit(chald2['index'])
# lb.classes_
# #output_lab = lb.transform(output_index)


array([   0,    1,    2, ..., 5603, 5604, 5605])

### Obtain the rest of the features

#### For TRAINING SET

In [10]:
def feature_extraction(chald2_col, Chin, Chinvalid):
    #print chald2_col
    ## creat the input array with the counts of languages
    tmp1 = Chin.apply([lambda y: chald2_col.loc[y]])
    tmp1.columns = tmp1.columns.droplevel()
    tmp1.head()
    lan_count_in = tmp1.apply(pd.Series.value_counts,axis=1)
    lan_count_in[lan_count_in.isna()]=0

    lan_count_in_t=lan_count_in.copy()
    lan_count_in_t = pd.concat([lan_count_in_t]*3)
    lan_count_in_t= lan_count_in_t.sort_index()

    tmp3 = Chinvalid.apply([lambda y: chald2_col.loc[y]])
    tmp3.columns = tmp3.columns.droplevel()
    # print tmp3.head()
    lan_count_in_val = tmp3.apply(pd.Series.value_counts,axis=1)
    lan_count_in_val[lan_count_in_val.isna()]=0
    print lan_count_in_val.shape
    lan_count_in_val.head()
#     ## creat the input array with the counts of languages
#     %time tmp2 = Chout.apply([lambda y: chald2.programming_language.loc[y]])
#     tmp2.columns = tmp2.columns.droplevel()
#     tmp2.head()
#     %time lan_count_out = tmp2.apply(pd.Series.value_counts,axis=1)
#     lan_count_out[lan_count_out.isna()]=0

    tmo = lan_count_in_val
    print lan_count_in_val.shape
    print tmo.shape
    tmo2 = tmo.append(lan_count_in_t.iloc[1,:])
    print tmo2.shape
    lan_count_in_val = tmo2[lan_count_in.columns.values]
    print lan_count_in_val.shape
    lan_count_in_val = lan_count_in_val[:-1]
    print lan_count_in_val.shape
    return lan_count_in_t, lan_count_in_val

# tmp1, tmp2 = feature_extraction(chald2_col, Chin)
chald2_col = chald2.programming_language
lan_in_f, lan_val_f = feature_extraction(chald2.programming_language, Chin, Chinvalid)

(39732, 3)
(39732, 3)
(39732, 3)
(39733, 3)
(39733, 3)
(39732, 3)


In [11]:
chald2.head()
chald2_col = chald2.challenge_series_ID
chal_in_f, chal_val_f = feature_extraction(chald2_col, Chin, Chinvalid)

(39732, 392)
(39732, 392)
(39732, 392)
(39733, 426)
(39733, 419)
(39732, 419)


In [12]:
chald2.head()
chald2_col = chald2.category_id
cat_in_f, cat_val_f = feature_extraction(chald2_col, Chin, Chinvalid)

(39732, 177)
(39732, 177)
(39732, 177)
(39733, 187)
(39733, 184)
(39732, 184)


In [13]:
chald2.head()
chald2_col = chald2.author_org_ID
aoid_in_f, aoid_val_f = feature_extraction(chald2_col, Chin, Chinvalid)

(39732, 1412)
(39732, 1412)
(39732, 1412)
(39733, 1620)
(39733, 1537)
(39732, 1537)


In [14]:
chald2.head()
chald2_col = chald2.author_gender
gen_in_f, gen_val_f = feature_extraction(chald2_col, Chin, Chinvalid)

(39732, 3)
(39732, 3)
(39732, 3)
(39733, 3)
(39733, 3)
(39732, 3)


### Put multiple (two) different INPUT feature sets together

#### For the TRAINING SET

In [15]:
# chal_id_in.head()
df_in = {'ABC' : lan_in_f, 'XYZ' : chal_in_f, 'abc' : cat_in_f, 'abc2': gen_in_f, 'abc3': aoid_in_f}
df_in = pd.concat(df_in.values(),axis=1,keys=df_in.keys())
print df_in.shape
df_in.head()



(208596, 2146)


XYZ                                                        \
        0.0      2435.0   2436.0   2439.0   2440.0   2441.0   2442.0     
user_id                                                                  
4576         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4576         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4576         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4580         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4580         0.0      0.0      0.0      0.0      0.0      0.0      0.0   

                                     ...        abc3                    \
        2443.0   2444.0   2445.0     ...    101702.0 101703.0 101704.0   
user_id                              ...                                 
4576         0.0      0.0      0.0   ...         0.0      0.0      0.0   
4576         0.0      0.0      0.0   ...         0.0      0.0      0.0   
4576         0.0      0.0      0.0   ...         0.0      0.0      0.0   
4580         0.0      0.0      0.0   ...         0.0      0.0      0.0   
4580         0.0      0.0      0.0   ...         0.0      0.0      0.0   

                                                                        
        101705.0 101707.0 101708.0 101711.0 101715.0 101717.0 101718.0  
user_id                                                                 
4576         0.0      0.0      0.0      0.0      0.0      0.0      0.0  
4576         0.0      0.0      0.0      0.0      0.0      0.0      0.0  
4576         0.0      0.0      0.0      0.0      0.0      0.0      0.0  
4580         0.0      0.0      0.0      0.0      0.0      0.0      0.0  
4580         0.0      0.0      0.0      0.0      0.0      0.0      0.0  

[5 rows x 2146 columns]

#### For the VALIDATION SET

In [16]:
# chal_id_in.head()
# lan_in_f, lan_val_f
# chal_in_f, chal_val_f
print lan_val_f.shape
print chal_val_f.shape
df_in_val = {'ABC' : lan_val_f, 'XYZ' : chal_val_f, 'abc' : cat_val_f, 'abc2' : gen_val_f, 'abc3' : aoid_val_f}
df_in_val = pd.concat(df_in_val.values(),axis=1,keys=df_in_val.keys())
print df_in_val.shape
print Chinvalid.shape
df_in_val.head()



(39732, 3)
(39732, 419)
(39732, 2146)
(39732, 1)


XYZ                                                        \
        0.0      2435.0   2436.0   2439.0   2440.0   2441.0   2442.0     
user_id                                                                  
4577         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4578         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4579         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4583         0.0      0.0      0.0      0.0      0.0      0.0      0.0   
4584         0.0      0.0      0.0      0.0      0.0      0.0      0.0   

                                     ...        abc3                    \
        2443.0   2444.0   2445.0     ...    101702.0 101703.0 101704.0   
user_id                              ...                                 
4577         0.0      0.0      0.0   ...         0.0      0.0      0.0   
4578         0.0      0.0      0.0   ...         0.0      0.0      0.0   
4579         0.0      0.0      0.0   ...         0.0      0.0      0.0   
4583         0.0      0.0      0.0   ...         0.0      0.0      0.0   
4584         0.0      0.0      0.0   ...         0.0      0.0      0.0   

                                                                        
        101705.0 101707.0 101708.0 101711.0 101715.0 101717.0 101718.0  
user_id                                                                 
4577         0.0      NaN      NaN      NaN      NaN      NaN      NaN  
4578         0.0      NaN      NaN      NaN      NaN      NaN      NaN  
4579         0.0      NaN      NaN      NaN      NaN      NaN      NaN  
4583         0.0      NaN      NaN      NaN      NaN      NaN      NaN  
4584         0.0      NaN      NaN      NaN      NaN      NaN      NaN  

[5 rows x 2146 columns]

# Fit input and output to a model

### Split to test and train set

#### For TRAINING SET

In [17]:
X = df_in.copy()#np.random.randint(5, size=(6, 100))
# just in case, but should have gotten rid of these values already
X[X.isna()]=0
X = X.astype(int)
X = X.as_matrix(columns=None)
print X.shape

# Binarized output labels (need to take its transpose)
# y = output_lab.T
# print y.shape

# non-binarized output labels
y = output_index
y = y.astype(int)
y = y.as_matrix(columns=None)

(208596, 2146)


#### For VALIDATION SET

In [18]:
Xtest = df_in_val.copy()#np.random.randint(5, size=(6, 100))
# just in case, but should have gotten rid of these values already
Xtest[Xtest.isna()]=0
Xtest = Xtest.astype(int)
Xtest = Xtest.as_matrix(columns=None)
print Xtest.shape

(39732, 2146)


#### actually split the training data

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [20]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape




(139759, 2146)
(68837, 2146)
(139759,)
(68837,)


### Multinomial Naive Bayes

#### Trained on part of the training set (cross-validation)

In [62]:
## Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(X_train, y_train)
%time MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
print(X_test)
%time clf.predict(X_test)
clf.fit(X_train, y_train)
sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X_test, y_test)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

CPU times: user 13 µs, sys: 8 µs, total: 21 µs
Wall time: 19.1 µs
[[0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]
CPU times: user 1min 30s, sys: 2.76 s, total: 1min 33s
Wall time: 16.2 s
Sparsity with L1 penalty: 0.00%
Test score with L1 penalty: 0.0443


In [ ]:
from sklearn.neural_network import MLPClassifier
clf_nn = MLPClassifier(solver='lbfgs', alpha=1e-3,hidden_layer_sizes=(10, 10), random_state=1)
clf_nn.fit(X_train, y_train)
%time clf_nn.predict(X_test)
#clf.fit(X_train, y_train)
sparsity = np.mean(clf_nn.coef_ == 0) * 100
score = clf_nn.score(X_test, y_test)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)


In [ ]:
clf_nn.predict(X_test)

#### Trained on the whole training set, run on the testing set

In [44]:
## Multinomial Naive Bayes
from sklearn.naive_bayes import MultinomialNB
clf_f = MultinomialNB()
clf_f.fit(X, y)
MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
# print(X_test)
# clf.predict(X_test)
# clf.fit(X_train, y_train)
# sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X, y)
# print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

Test score with L1 penalty: 0.0438


## Logistic Regression

#### Trained on part of the training set (cross-validation)

In [ ]:
# Multinomial Logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = scaler.fit_transform(X)
# X_test = scaler.transform(X_test)

# Turn up tolerance for faster convergence
clf_log = LogisticRegression(C=50. / len(X),
                         multi_class='multinomial',
                         penalty='l1', solver='saga', tol=0.1)
clf_log.fit(X, y)
# sparsity = np.mean(clf.coef_ == 0) * 100
score = clf.score(X, y)
print("Sparsity with L1 penalty: %.2f%%" % sparsity)
print("Test score with L1 penalty: %.4f" % score)

#### Make the prediciton

In [49]:
tmp = clf_f.predict_log_proba(Xtest)
%time tmp = pd.DataFrame(tmp).apply(lambda row: [row.nlargest(3).index.get_values()], axis=1)
a = pd.DataFrame(tmp)
a = a.apply(lambda row: row[0],axis=1)
s = a.apply((lambda x: pd.Series(x[0])),axis=1).stack().reset_index(level=1, drop=True)
s.head()

CPU times: user 22.4 s, sys: 99.6 ms, total: 22.5 s
Wall time: 22.5 s


0    931
0    940
0    953
1    931
1    940
dtype: int64

## Create output file

In [50]:
print s.shape
submiss = samp_sub.copy()
print submiss.head()
print submiss.shape
submiss['challenge'] = chald2['index'].index[s].get_values()
submiss.head()

submiss.to_csv('MultinomialNB_smallest.csv',index=False)

(119196,)
  user_sequence challenge
0       4577_11   CI23648
1       4577_12   CI23648
2       4577_13   CI23648
3       4578_11   CI23648
4       4578_12   CI23648
(119196, 2)


In [575]:
chald2['index'].index[s].get_values().shape

(119199,)

In [576]:
submiss['challenge'].shape

(119196,)

In [340]:
dftest.head(14)

,user_sequence,user_id,challenge_sequence,challenge
0,4577_1,4577,1,CI23855
1,4577_2,4577,2,CI23933
2,4577_3,4577,3,CI24917
3,4577_4,4577,4,CI24915
4,4577_5,4577,5,CI23714
5,4577_6,4577,6,CI23663
6,4577_7,4577,7,CI24958
7,4577_8,4577,8,CI25135
8,4577_9,4577,9,CI25727
9,4577_10,4577,10,CI24530


In [304]:
dfte = pd.DataFrame(dftest.groupby(['user_id'])['challenge'].apply(list))
Chtein = pd.DataFrame(dfte.apply(lambda y: y.iloc[0][0:10],axis=1))

## Nearest 3 challenge for a given vector (Not Used in the end)

In [ ]:
# submiss = samp_sub.copy()
# # submiss = submiss
# # print submiss
# df = np.empty(0)
# for k in range(len(submiss)/3):
#     predic = chald2.iloc[k]
#     predic.total_submissions = 23
# # print (predic.iloc[0] == chald2.iloc[:5]).sum(axis=1).nlargest(3).index.get_values()
# # predic.apply((lambda row: (row == chald2.iloc[:5]).sum(axis=1).nlargest(3).index.get_values()),raw=True)
# # out.head()
#     ID_pred = (chald2 == predic).sum(axis=1).nlargest(3)
#     df = np.append(df, ID_pred.index.get_values())
# print len(submiss)/3
# print k
# print df
# submiss['challenge'] = df
# submiss.head()
# # chald2.iloc[:500] == predic

In [85]:
submiss = samp_sub.copy()
submiss = submiss.iloc[:10]
# print submiss
df = np.empty(0)

predic = chald2.iloc[:5]


# compare the predicted values to the nearest element from all the challenges
out = predic.apply((lambda row: (row == chald2).sum(axis=1).nlargest(3)),axis=1)
out[out.notna()]
cc = out.columns.values

out2 = pd.DataFrame(out.apply((lambda row: [cc[row.notna()]]),axis=1))
out2 = out2.apply((lambda row: row[0]),axis=1)
s = out2.apply(lambda x: pd.Series(x[0]),axis=1).stack().reset_index(level=1, drop=True)
print s.shape
submiss.challenge = s.reset_index()
submiss.head(15)



(15,)


,user_sequence,challenge
0,4577_11,CI23478
1,4577_12,CI23478
2,4577_13,CI23478
3,4578_11,CI23479
4,4578_12,CI23479
5,4578_13,CI23479
6,4579_11,CI23480
7,4579_12,CI23480
8,4579_13,CI23480
9,4583_11,CI23481


In [305]:
for k in range(len(Chtein)):
    if chald2.programming_language.loc[Chtein.iloc[k][0]].unique().size != 1:        

        print chald2.programming_language.loc[Chtein.iloc[k][0]].unique()
        #print chald2.programming_language.loc[Chout.iloc[k][0]].unique()
        print k

[2 1]
1711
[1 2]
2240
[1 2]
2406
[2 1]
3029
[2 1]
4212
[2 1]
4215
[2 1]
5553
[2 1]
5728
[1 2]
5931
[3 1]
5986
[3 1]
6140
[2 1]
6215
[2 1]
6240
[2 1]
6251
[1 3]
6337
[1 2]
6401
[3 1]
6517
[1 2]
6747
[1 2]
7410
[2 1]
7857
[1 3]
7878
[1 3]
8067
[2 1]
8284
[3 1]
8305
[3 1]
8372
[1 2]
8384
[1 2]
8505
[1 2]
8602
[2 1]
8604
[3 1]
8611
[3 1]
9171
[2 1]
9188
[2 1]
9230
[2 1]
9277
[3 1]
9411
[2 1]
9598
[1 2]
9770
[1 2]
9771
[1 2]
9822
[1 2]
9986
[1 2]
10027
[1 2]
10060
[2 1]
10594
[1 2]
10721
[1 2]
10835
[3 1]
10843
[2 1]
10925
[2 1]
11015
[3 1]
11276
[3 1]
11347
[1 2]
11527
[2 1]
11627
[1 2]
11758
[3 1]
11827
[1 2]
11910
[3 1]
11984
[2 1]
12182
[3 1]
12355
[3 1]
12379
[1 3]
12423
[1 3]
12482
[2 3 1]
12615
[1 2]
12635
[1 2]
12892
[3 1]
12930
[3 1]
12933
[2 1]
13066
[1 2]
13090
[2 1]
13192
[1 2]
13247
[1 2]
13277
[1 2]
13363
[3 1]
13476
[2 1 3]
13608
[1 2]
13695
[1 2]
13736
[2 1]
13790
[1 2]
13805
[1 2]
14004
[1 2]
14093
[1 2]
14126
[2 1]
14152
[3 1]
14191
[3 1]
14240
[2 1]
14314
[1 2]
14382
[1 2

In [97]:
samp_sub.head()

,user_sequence,challenge
0,4577_11,CI23648
1,4577_12,CI23648
2,4577_13,CI23648
3,4578_11,CI23648
4,4578_12,CI23648


In [122]:
#dftest.user_id.unique()
from random import randint
submiss = samp_sub.copy()
for i in range(5):#len(submiss.challenge)):
     print chald.challenge_ID[randint(0,5606)]
     submiss.challenge[i] = chald.challenge_ID[randint(0,5605)]
submiss.head()



CI23582
CI24086
CI27886
CI23920
CI26835


In [108]:
## print the predicted challenges to a dataset
submiss.to_csv('submission.csv',index=False)

238392

In [109]:
samp_sub.size

238392

69532